In [ ]:
! pip install -U langchain openai langchain-experimental # (newest versions required for multi-modal)
! pip install "unstructured[all-docs]" pillow pydantic lxml pillow matplotlib chromadb tiktoken


In [ ]:
# !pip install unstructured

In [ ]:
!pip show unstructured

In [ ]:
import glob
import pandas as pd
from langchain.document_loaders import UnstructuredFileLoader
from openai import AzureOpenAI
import time

folder_path="/kaggle/input/equity-research/"
number_chunck_selected_for_each_file=100


def get_pdf_file_paths(folder_path):
    pdf_files = glob.glob(folder_path + "/*.pdf")
    return pdf_files



def pdf_to_chunck_list(result1,path):
    result=[]
    loader = UnstructuredFileLoader(
    path, mode="elements"
  )
    docs = loader.load()
    for i in range(len(docs)):
        if docs[i].metadata["category"] in ["UncategorizedText","Footer","Header","Title"]:
            pass
        elif "Disclosures" in docs[i].page_content.cotains():
            break
        else:
            result.append(docs[i].page_content)
    
    for i in range(len(result)):
        result1.append(result[i])
    return result1

def prepare_dataframe(folder_path):
    pdf_file_paths = get_pdf_file_paths(folder_path)
    result=[]
    for i in pdf_file_paths:
        result=pdf_to_chunck_list(result,i)
    df=pd.DataFrame({"input":result})
    empty_mask = df["input"] == ""
    df = df[~empty_mask]
    empty_mask = df["input"].isnull()
    df=df[~empty_mask]
    empty_mask = df["input"].isna()
    df=df[~empty_mask]
    filtered_rows = df[df['input'].str.split(" ").str.len() < 20]
    df = df.drop(filtered_rows.index)
    df = df.reset_index(drop=True)
    return df




def question_generate_from_llm(df):
    error = None
    api_key = "ebc76beedf89486382e773d3b4cf0b10"
    client = AzureOpenAI(
        api_key=api_key,
        api_version="2023-12-01-preview",
        azure_endpoint="https://gpgpt.openai.azure.com/"
    )
    result = []
    token_count = 0
    try:
        for index, row in df.iterrows():
            prompt = "<text>" + row['input'] + "</text><task>this text is the exam question answer, you need to generate the question based on it </task>"
            message = [
                {"role": "system", "content": "you are the CFA equity Investment examiner"},
                {"role": "user", "content": prompt}
            ]
            token_count = len(prompt.split(" ")) + token_count

            if token_count > 3000:
                time.sleep(5)
                token_count = 0

            while True:
                try:
                    completion = client.chat.completions.create(
                        model="test",
                        messages=message
                    )
                    result.append(completion.choices[0].message.content)
                    print("current index:", index)
                    break
                except Exception as e:
                    error = e
                    # Handle rate limit exceeded error
                    if "code" in e.error or e.error["code"] == 429:
                        # Sleep for 60 seconds and retry after rate limit reset
                        time.sleep(60)
                        print("Error:", e)
                        print("Current index:", index)
                        break
                    else:
                        # Handle other exceptions
                        print("Error:", e)
                        break

            if error is not None:
                print("Current index of the DataFrame is:", index)
    except KeyboardInterrupt:
        print("Execution interrupted.")

    df=pd.DataFrame({"result":result})
    df.to_csv("backup.csv")

    return result

def merge_llm_result_to_dataframe(result,df):
    df.rename(columns={'input': 'output'}, inplace=True)
    df['input']=result
    return df



# if __name__ == '__main__':
#   df=prepare_dataframe(folder_path)
#   print("finish prepare datafraome")
#   result=question_generate_from_llm(df)
#   print("finish generate from llm")
#   length=len(result)
#   df=merge_llm_result_to_dataframe(result,df[:length])
#   df.to_csv("dataset.csv")








In [ ]:
df=prepare_dataframe(folder_path)

# data=question_generate_from_llm(df)


In [ ]:
data=question_generate_from_llm(df)
data_output=merge_llm_result_to_dataframe(data,df)
data_output.to_csv("cfa-three-chapter.csv")

In [ ]:
# error=None
# api_key="ebc76beedf89486382e773d3b4cf0b10"

# client = AzureOpenAI(
#       api_key=api_key,
#       api_version="2023-12-01-preview",
#       azure_endpoint="https://gpgpt.openai.azure.com/"
#   )

In [ ]:
# for i in range(5):
#   prompt = "<text>" + df.iloc[i]['input'] + "</text><task>this text is the exam question answer, you need to generate the question based on it </task>"
#   message = [
#                 {"role": "system", "content": "you are the CFA equity Investment examiner"},
#                 {"role": "user", "content": prompt}
#             ]
#   completion = client.chat.completions.create(
#                       model="test",
#                       messages=message
#                   )
#   print(completion.choices[0].message.content)





In [ ]:
# length=len(data)
# data_copy=data
# data_output=merge_llm_result_to_dataframe(data_copy,df[:length])
# data_output

In [ ]:
# # data_output=data_output[['input','output']]
# filtered_rows = data_output[data_output['output'].astype(str).str.contains('disclose|Source', case=False)]
# data_output = data_output.drop(filtered_rows.index)
# data_output.to_csv("data_output_result.csv")

In [ ]:
# filtered_rows = data_output[data_output['output'].str.split(" ").str.len() < 20]
# data_output = data_output.drop(filtered_rows.index)
# data_output.to_csv("data_output_result.csv")

In [ ]:
# data_output = data_output.reset_index(drop=True)
# data_output.to_csv("data_output_result.csv")

In [ ]:
# filtered_rows.to_csv("check.csv")

In [ ]:
# loader = UnstructuredFileLoader(
#     '/kaggle/input/cfa-three-chapter/Financial_statement.pdf', mode="elements"
#   )
# docs = loader.load()
# result1=[]
# result2=[]
# for i in range(len(docs)):
#     result1.append(docs[i].metadata["category"])
#     result2.append(docs[i].page_content)



In [ ]:
# result=pd.DataFrame({'category':result1,'text':result2})

In [ ]:
# result["category"].value_counts()


In [ ]:
# filtered_result = result[(result['category'] == 'ListItem') & (result['text'].str.split(" ").str.len() > 6)]

In [ ]:
# filtered_result = result[(result['category'] == 'ListTitle') & (result.index.to_series().diff() == 1)]


In [ ]:
# result[result['category']=="UncategorizedText"]